In [4]:
import pandas as pd
import numpy as np
import re 


# Функция для загрузки графиков

In [3]:
def holyday(data, day, uuid, day_start, day_end, over_200 = np.nan):
    """Функция получается на входе предобработанный датасет, список дней, дату начала и окончания  в формате '2025-05-01' 
       и если нужна отсечка для городов с начеление больше 200к отсечку в формате '18:00'. На выходе получается файл для передачи в ТП  """
    data['worktime'] = ''
    for i in range(len(data)):
        if data.loc[i, 'schedule_1_day'] in day:
            data.loc[i, 'worktime'] = data.loc[i, 'schedule_1_time']
        elif data.loc[i, 'schedule_2_day'] in day:
            data.loc[i, 'worktime'] = data.loc[i, 'schedule_2_time']
        elif data.loc[i, 'schedule_3_day'] in day:
            data.loc[i, 'worktime'] = data.loc[i, 'schedule_3_time']
        else:
            data.loc[i, 'is_holiday'] = 'True'

    data = data.merge(uuid, on='system_name', how = 'left')

    data['worktime_begin'] = data['worktime'].str.split('-').str.get(0)
    data['worktime_end'] = data['worktime'].str.split('-').str.get(1)
    data = data.drop('worktime', axis=1)
    data['scheduled_date_begin'] = day_start
    data['scheduled_date_end'] = day_end
    data['comment'] = 'Праздничный день'
    if 'is_holiday' in data.columns:
        pass
    else:
        data['is_holiday'] = '' 
    try:
        data.loc[data['worktime_end'] > over_200, 'worktime_end'] = over_200
    except:
        pass

    
    data = data[['system_name', 'Офис uuid', 'scheduled_date_begin', 'scheduled_date_end','is_holiday', 'worktime_begin', 'worktime_end','comment']]
    
    return data


# Датасеты

___Загружаем датасеты___

In [16]:
data = pd.read_excel('Downloads/office-23-04-2025--12_43.xls')

In [41]:
over_200_k = pd.read_excel('Downloads/свыше_200к.xlsx')

over_200_k.loc[99] = "Иркутск"

over_200_k['Город'] = over_200_k['Город'].str.lower()

___Добавляем города, прописываем население городов___

In [42]:
data["over_200"] = ""

In [43]:
data['city'] =''

In [44]:
for i in range(len(data)):
    k = data.loc[i,'Город'].split(", ")
    data.loc[i,'city'] = k[0]
    

In [45]:
data['city'] = data['city'].str.lower()

In [46]:
over_200_unique = list(over_200_k['Город'].unique())

In [47]:
for i in range(len(data)):
    if data.loc[i, "city"].strip() in over_200_unique:
        data.loc[i, "over_200"] = "yes"
    else:
        data.loc[i, "over_200"] = "no"

___Присваеваем городам категорию в зависимости от населения___

In [48]:
#data['category'] = ''

"""for i in range(len(data)):
    if data.loc[i, 'over_200'] == 'yes':
        data.loc[i, 'category'] = "200+"
    elif  data.loc[i, 'Районный центр'] == 'Да':
        data.loc[i, 'category'] = "<50"
    else:
        data.loc[i, 'category'] = "50-200"
        """

'for i in range(len(data)):\n    if data.loc[i, \'over_200\'] == \'yes\':\n        data.loc[i, \'category\'] = "200+"\n    elif  data.loc[i, \'Районный центр\'] == \'Да\':\n        data.loc[i, \'category\'] = "<50"\n    else:\n        data.loc[i, \'category\'] = "50-200"\n        '

In [49]:
data.drop(columns = ['Наименование (Новое)','Районный центр','Город'], inplace=True)

In [50]:
#data  = data[['city', 'Наименование', 'Код', 'category', 'График работы', 'Тип офиса']]
data  = data[['city', 'Наименование', 'Код', 'График работы', 'Тип офиса', 'over_200']]

In [51]:
data.rename(columns = {'Код':'system_name', 'График работы': 'schedule', 'Наименование':'name', 'Тип офиса':'type'}, inplace=True)

In [52]:
data['schedule_1']=data['schedule'].str.split(', ').str.get(0)
data['schedule_2']=data['schedule'].str.split(', ').str.get(1)
data['schedule_3']=data['schedule'].str.split(', ').str.get(2)

In [53]:
try:
    data['schedule_3_day']=data['schedule_3'].str.split(' ').str.get(0)
    data['schedule_3_time']=data['schedule_3'].str.split(' ').str.get(1)
    data['schedule_1_day'] = data['schedule_1'].str.split(' ').str.get(0)
    data['schedule_1_time'] = data['schedule_1'].str.split(' ').str.get(1)
    data['schedule_2_day']=data['schedule_2'].str.split(' ').str.get(0)
    data['schedule_2_time']=data['schedule_2'].str.split(' ').str.get(1)
except:
    data['schedule_1_day'] = data['schedule_1'].str.split(' ').str.get(0)
    data['schedule_1_time'] = data['schedule_1'].str.split(' ').str.get(1)
    data['schedule_2_day']=data['schedule_2'].str.split(' ').str.get(0)
    data['schedule_2_time']=data['schedule_2'].str.split(' ').str.get(1)

In [54]:
try:
    data.drop(columns=['schedule_1', 'schedule_2','schedule_3'], inplace=True)
except:
    data.drop(columns=['schedule_1', 'schedule_2'], inplace=True)

In [55]:
try:
    print(data['schedule_3_day'].unique())
    print(data['schedule_1_day'].unique())
    print(data['schedule_2_day'].unique())
except:
    print(data['schedule_1_day'].unique())
    print(data['schedule_2_day'].unique())

['Пн-Пт' 'Пн-Чт' 'Пн-Вс']
['Сб-Вс' 'Пт' nan]


# Майские праздники РФ 

In [67]:
# создаем несколько копий 

may_01 = data.copy()
may_01 = may_01.reset_index(drop=True)

may_02 = data.copy()
may_02 = may_02.reset_index(drop=True)

may_03 = data.copy()
may_03 = may_03.reset_index(drop=True)

may_04 = data.copy()
may_04 = may_04.reset_index(drop=True)

may_08 = data.copy()
may_08 = may_08.reset_index(drop=True)

may_09 = data.copy()
may_09 = may_09.reset_index(drop=True)

may_10 = data.copy()
may_10 = may_10.reset_index(drop=True)

may_11 = data.copy()
may_11 = may_11.reset_index(drop=True)

uuid = pd.read_csv('/Users/CDEK/Downloads/20250417_075038.csv', sep = ';')
uuid.rename(columns={'Системное наименование офиса':'system_name','office_uuid':'Офис uuid'}, inplace=True)

In [68]:
may_01 = holyday(may_01, ['Пн-Вс', 'Сб-Вс', 'Вс', 'Вт-Вс'], uuid, '2025-05-01', '2025-05-01', over_200 = '18:00')

may_02 = holyday(may_02, ['Пн-Вс', 'Пн-Сб', 'Вт-Сб', 'Сб-Вс', 'Сб', 'Вт-Вс'], uuid, '2025-05-02', '2025-05-02', over_200 = '18:00')

may_03 = holyday(may_03, ['Пн-Вс', 'Пн-Сб', 'Вт-Сб', 'Сб-Вс', 'Сб', 'Вт-Вс'], uuid, '2025-05-03', '2025-05-03', over_200 = '18:00')

may_04 = holyday(may_04, ['Пн-Вс', 'Сб-Вс', 'Вс', 'Вт-Вс'], uuid, '2025-05-04', '2025-05-04', over_200 = '18:00')

may_08 = holyday(may_08, ['Пн-Вс', 'Пн-Сб', 'Вт-Сб', 'Сб-Вс', 'Сб', 'Вт-Вс'], uuid, '2025-05-08', '2025-05-08', over_200 = '18:00')

may_09 = holyday(may_09, ['efwf', 'wefwe'], uuid, '2025-05-09', '2025-05-09', over_200 = '18:00')

may_10 = holyday(may_10, ['Пн-Вс', 'Пн-Сб', 'Вт-Сб', 'Сб-Вс', 'Сб', 'Вт-Вс'], uuid, '2025-05-10', '2025-05-10', over_200 = '18:00')

may_11 = holyday(may_11, ['Пн-Вс', 'Сб-Вс', 'Вс', 'Вт-Вс'], uuid, '2025-05-11', '2025-05-11', over_200 = '18:00')


In [89]:
with pd.ExcelWriter('may_russia.xlsx') as writer:  
    april_29.to_excel(writer, sheet_name=april_29.loc[2, 'scheduled_date_begin'], index = False)
    may_01.to_excel(writer, sheet_name=may_01.loc[2, 'scheduled_date_begin'], index = False)
    may_02.to_excel(writer, sheet_name=may_02.loc[2, 'scheduled_date_begin'], index = False)
    may_03.to_excel(writer, sheet_name=may_03.loc[2, 'scheduled_date_begin'], index = False)
    may_04.to_excel(writer, sheet_name=may_04.loc[2, 'scheduled_date_begin'], index = False)
    may_08.to_excel(writer, sheet_name=may_08.loc[2, 'scheduled_date_begin'], index = False)
    may_09.to_excel(writer, sheet_name=may_09.loc[2, 'scheduled_date_begin'], index = False)
    may_10.to_excel(writer, sheet_name=may_10.loc[2, 'scheduled_date_begin'], index = False)
    may_11.to_excel(writer, sheet_name=may_11.loc[2, 'scheduled_date_begin'], index = False)

# Майские праздники РБ 

In [310]:
april_29 = data.copy()
april_29 = april_29.reset_index(drop=True)

may_01 = data.copy()
may_01 = may_01.reset_index(drop=True)

may_08 = data.copy()
may_08 = may_08.reset_index(drop=True)

may_09 = data.copy()
may_09 = may_09.reset_index(drop=True)

In [312]:
april_29 = holyday(april_29, ['efwf', 'wefwe'], uuid, '2025-04-29', '2025-04-29')

may_01 = holyday(may_01, ['Пн-Вс', 'Сб-Вс', 'Вс'], uuid, '2025-05-01', '2025-05-01')

may_08 = holyday(may_08, ['Пн-Вс','Пн-Сб', 'Сб-Вс', 'Вт-Сб', 'Сб'], uuid, '2025-05-08', '2025-05-08')

may_09 = holyday(may_09, ['efwf', 'wefwe'], uuid, '2025-05-09', '2025-05-09')


In [313]:
with pd.ExcelWriter('may_bel.xlsx') as writer:  
    april_29.to_excel(writer, sheet_name=april_29.loc[2, 'scheduled_date_begin'], index = False)
    may_01.to_excel(writer, sheet_name=may_01.loc[2, 'scheduled_date_begin'], index = False)
    may_08.to_excel(writer, sheet_name=may_08.loc[2, 'scheduled_date_begin'], index = False)
    may_09.to_excel(writer, sheet_name=may_09.loc[2, 'scheduled_date_begin'], index = False)

# Майские праздники Казахстан

In [345]:
may_01 = data.copy()
may_01 = may_01.reset_index(drop=True)

may_07 = data.copy()
may_07 = may_07.reset_index(drop=True)

may_09 = data.copy()
may_09 = may_09.reset_index(drop=True)

In [349]:
may_01 = holyday(may_01, ['Пн-Вс', 'Пн-Сб', 'Сб', 'Сб-Вс'], uuid, '2025-05-01', '2025-05-01')

may_07 = holyday(may_07, ['Пн-Вс', 'Пн-Сб', 'Сб', 'Сб-Вс'], uuid, '2025-05-07', '2025-05-07')

may_09 = holyday(may_09, ['efwf', 'wefwe'], uuid, '2025-05-09', '2025-05-09')

In [354]:
with pd.ExcelWriter('may_kz.xlsx') as writer:  
    may_01.to_excel(writer, sheet_name=may_01.loc[2, 'scheduled_date_begin'], index = False)
    may_07.to_excel(writer, sheet_name=may_07.loc[2, 'scheduled_date_begin'], index = False)
    may_09.to_excel(writer, sheet_name=may_09.loc[2, 'scheduled_date_begin'], index = False)

# Ставрополь/Севастополь

In [43]:
april_29 = data.copy()

uuid = pd.read_csv('/Users/CDEK/Downloads/20250417_075038.csv', sep = ';')
uuid.rename(columns={'Системное наименование офиса':'system_name','office_uuid':'Офис uuid'}, inplace=True)

In [70]:
print(data['schedule_1_day'].unique())
print(data['schedule_2_day'].unique())


['Пн-Пт' 'Пн-Вс' 'Пн-Сб']
['Сб-Вс' nan 'Вс']


In [44]:
april_29 = holyday(april_29, ['Пн-Вс', 'Пн-Сб','Сб-Вс'], uuid, '2025-04-29', '2025-04-29')

april_29['worktime_begin'] = '14:00'

In [71]:
april_29_irk = data.copy()

april_29_irk = holyday(april_29_irk, ['Пн-Вс', 'Пн-Сб','Сб-Вс'], uuid, '2025-04-29', '2025-04-29')

In [74]:
april_29_full = pd.concat([april_29, april_29_irk], ignore_index=False)

In [75]:
april_29_full

,system_name,Офис uuid,scheduled_date_begin,scheduled_date_end,is_holiday,worktime_begin,worktime_end,comment
0,NVM1,e7e9ce85-3c6f-4a1f-a9a0-2974d8e94de7,2025-04-29,2025-04-29,,14:00,19:00,Праздничный день
1,SVT6,37a76b34-634a-4915-972e-f271257a27ad,2025-04-29,2025-04-29,,14:00,20:00,Праздничный день
2,PTG3,9b874dc0-4874-446b-bcde-7ad673c830e0,2025-04-29,2025-04-29,,14:00,20:00,Праздничный день
3,ZKN1,d7290be6-12b8-4b96-9538-2a96e0e397ac,2025-04-29,2025-04-29,,14:00,19:00,Праздничный день
4,STV7,749e2f07-e4b0-4ff6-935e-6d6565549eef,2025-04-29,2025-04-29,,14:00,20:00,Праздничный день
...,...,...,...,...,...,...,...,...
37,IRK30,f0f585c1-f288-4f17-82c1-2d4462cc8822,2025-04-29,2025-04-29,,10:00,18:00,Праздничный день
38,IRK6,f3e02d5c-43aa-4d32-b3d3-3d827a1015cd,2025-04-29,2025-04-29,,10:00,18:00,Праздничный день
39,IRK88,f873ab28-154e-4eee-9234-0d93ed1aa36e,2025-04-29,2025-04-29,,10:00,18:00,Праздничный день
40,IRK98,fe8f8717-5b18-4fa2-a965-fc59e8a15d68,2025-04-29,2025-04-29,,10:00,18:00,Праздничный день


In [81]:
with pd.ExcelWriter('may_kz (1).xlsx') as writer:  
    april_29_full.to_excel(writer, sheet_name= '2025-04-29', index = False)